In [2]:
## DS 4300 Example - from docs
import re                   # Text preprocessing stuff
import string               # More text preprocessing
import nltk                 # Tokenization
import csv                  # CSV writing

import ollama               # Ollama
import redis                # Redis
import numpy as np          # Numpy
import fitz                 # PDF Reader

from tqdm import tqdm       # Progress bar bc I'm impatient
import os                   # Navigate folders
import time                 # Timing
import tracemalloc          # Memory Usage
import cpuinfo              # CPU Info
import psutil               # Memory Info

from sentence_transformers import SentenceTransformer       # Embedding Model
from collections import Counter                             # Simple counting dictionary
from redis.commands.search.query import Query               # Querying 
from redis.commands.search.field import VectorField, TextField

# Libraries for nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Important constants
VECTOR_DIM = 768
INDEX_NAME = "embedding_index"
DOC_PREFIX = "slides"
DISTANCE_METRIC = "COSINE"
QUERY = 'What is the CAP Theorem?'
LLM_MODEL = 'llama3.2:latest'
EMBEDDING_MODEL = 'sentence-transformers/all-mpnet-base-v2'

# Initialize Redis connection
redis_client = redis.Redis(host="localhost", port=6379, db=0, decode_responses=True)

def clear_redis_store():
    """
    Clears redis database store to prevent overlap

    Parameters:         C              A
        None           / \            1 C
                      A   3            2 3 
    Returns:         1 2                LL
        None
    """
    print("Clearing existing Redis store...")
    redis_client.flushdb()
    print("Redis store cleared.")


def create_hnsw_index():
    """
    Clears an HNSW index in the Redis database

    Parameters:          C                 C               B
        None           A   3            B     3         A     C
                      1 B             A   R             
    Returns:           L R           1 L  
        None
    """
    # Removes index if it already exists
    try:
        redis_client.execute_command(f"FT.DROPINDEX {INDEX_NAME} DD")
    except redis.exceptions.ResponseError:
        pass

    # Creates index
    redis_client.execute_command(
        f"""
        FT.CREATE {INDEX_NAME} ON HASH PREFIX 1 {DOC_PREFIX}
        SCHEMA text TEXT
        embedding VECTOR HNSW 6 DIM {VECTOR_DIM} TYPE FLOAT32 DISTANCE_METRIC {DISTANCE_METRIC}
        """
    )
    print("Index created successfully.")


def get_embedding(text: str, embedding_model) -> list:
    """
    Generate an embedding via a specified model.    B+ tree n-1 keys -> n children, default make nodes half full
                                                    Parents are lowest values of right n-1 keys
    Parameters:
        text (str): Text to embed
        embedding_model (str): Name of the embedding model to use

    Returns:
        response (np.Array): Numerical array representation of the embeddings
    
    """
    model = SentenceTransformer(embedding_model, trust_remote_code=True)
    response = model.encode(text)
    return response


def store_embedding(file: str, page: str, chunk: str, embedding: list):
    """
    Stores the embeddings in the Redis index        Hashtable: Apply hashing function to key, insert into hashed
                                                    index a tuple of (non-hash key, val), linked list if multiple
    Parameters:                                     lambda = num inserts / num hashes, should be < 0.9
        file (str): Name of the file for indexing
        page (str): Page number of the file
        chunk (str): Chunk number of the file
        embedding (list): Embedding representation of a single chunk

    Returns:
        None
    """

    key = f"{DOC_PREFIX}:{file}_page_{page}_chunk_{chunk}"
    redis_client.hset(
        key,
        mapping={
            "file": file,
            "page": page,
            "chunk": chunk,
            "embedding": np.array(
                embedding, dtype=np.float32
            ).tobytes(),  
        },
    )


def extract_text_from_pdf(pdf_path):
    """
    Extract text by page from the pdf               ACID: Atomicity, Consistency, Isolation, Durability
                                                        everything as unit, stable state to stable state, 
    Parameters:                                         transactions are independent, transactions are permamet
        pdf_path (str): Path to the pdf file            even at system failure

    Returns:
        text_by_page (list): List of the text on each page
    """

    doc = fitz.open(pdf_path)
    text_by_page = []
    for page_num, page in enumerate(doc):
        text_by_page.append((page_num, page.get_text()))
    return text_by_page


def preprocess_text(text):
    """
    Preprocesses and tokenizes text. Steps can be commented out if need be. 

    Transaction: CRUD operations performed as a single unit of work. All work or all abort
    Dirty read, Non-repeatable read, phantom read (reference data not existing)
    Scale up, then out by using distributed system that can withstand conflicts

    CAP: Systems are either consistent, available, or partition tolerant
    Identical views, present during failure, work in different segments

    Parameters:                                 
        text (str): Text to be tokenized

    Returns
        tokens (list): List of tokenized words
    """

    # Replace new lines
    text = text.replace('\n', ' ').strip()

    # Normalize case
    # text = text.lower()

    # Tokenization
    tokens = nltk.tokenize.word_tokenize(text)

    # Remove stopwords if need be
    # tokens = remove_stopwords(tokens)

    # Replaces wacky symbols (like stylized bullets) with <SYM> token if need be
    tokens = ["<SYM>" if re.fullmatch(r"[^\w\d" + re.escape(string.punctuation) + "]", token) else token for token in tokens]

    # Replaces words that show up only once with <UNK> token if need be
    # rare = [item[0] for item in Counter(tokens).items() if item[1] == 1]
    # tokens = ['<UNK>' if token in rare else token for token in tokens]

    # Replaces pure numbers with <NUM> token if need be
    # tokens = ['<NUM>' if token.isdigit() else token for token in tokens]

    # Removes punctuation marks
    # tokens = [token for token in tokens if token not in string.punctuation]


    # KV is very simple, quick o(1), horizontally scalable, example being redis
    # Easily store model features, intermediate results
    return tokens

# split the text into chunks with overlap
def split_text_into_chunks(text, chunk_size=300, overlap=50):
    """Split text into chunks of approximately chunk_size words with overlap.
    
    BASE: Basically available, soft state, eventual consistency
    Pessimistic: Something could wil go wrong
    Optimistic: If something happens, should be ok
    """
    words = preprocess_text(text)
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i : i + chunk_size])
        chunks.append(chunk)
    return chunks

# Process all PDF files in a given directory, returns elapsed time and peak memory
def process_pdfs(data_dir, model, chunk_size=300, overlap=50):
    """
    INCR somevalue
    INCRBY somevalue N
    SET _ N
    GET 

    Has hash and linked list structures
    Supports sets, json
    """

    # Start time / memory check
    tracemalloc.start()
    start_time = time.time()

    for file_name in tqdm(os.listdir(data_dir)):
        if file_name.endswith(".pdf"):
            pdf_path = os.path.join(data_dir, file_name)
            text_by_page = extract_text_from_pdf(pdf_path)
            for page_num, text in text_by_page:
                chunks = split_text_into_chunks(text, chunk_size, overlap)
                for chunk_index, chunk in enumerate(chunks):
                    embedding = get_embedding(chunk, model)
                    store_embedding(
                        file=file_name,
                        page=str(page_num),
                        chunk=str(chunk),
                        embedding=embedding,
                    )

    elapsed = time.time() - start_time
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f'Time elapsed: {round(elapsed, 4)} seconds')
    print(f"Peak memory usage: {peak / 1024**2:.2f} MiB")

    # returns time and peak memory
    return round(elapsed, 4), round((peak / 1024**2), 2)


def search_embeddings(query, model, top_k=3):
    """
    json lightweight, easy for humans and machines, many uses
    bson is binary serialization of json, allows working with more types + minimize space
    xml is older version like json, uses tags like html
    Doc datab is good for more oo design + complex structuers that are self describing
    """

    query_embedding = get_embedding(query, model)

    # Convert embedding to bytes for Redis search
    query_vector = np.array(query_embedding, dtype=np.float32).tobytes()

    try:
        q = (
            Query("*=>[KNN 5 @embedding $vec AS vector_distance]")
            .sort_by("vector_distance")
            .return_fields("id", "file", "page", "chunk", "vector_distance")
            .dialect(2)
        )

        # Perform the search
        results = redis_client.ft(INDEX_NAME).search(
            q, query_params={"vec": query_vector}
        )

        # Transform results into the expected format
        top_results = [
            {
                "file": result.file,
                "page": result.page,
                "chunk": result.chunk,
                "similarity": result.vector_distance,
            }
            for result in results.docs
        ][:top_k]

        # Print results for debugging
        # for result in top_results:
        #     print(
        #         f"---> File: {result['file']}, Page: {result['page']}, Chunk: {result['chunk']}"
        #     )

        return top_results

    except Exception as e:
        print(f"Search error: {e}")
        return []
    
def generate_rag_response(query, context_results, model):
    """
    MongoDB has lots of query support, idnexing, replication
    db.users.find({'key': 'value'})
    "imdb.rating": { $gte: 7 } 
    $eq, $gt, $ne, $in, $nin
    
    """

    # Prepare context string
    context_str = "\n".join(
        [
            f"From {result.get('file', 'Unknown file')} (page {result.get('page', 'Unknown page')}, chunk {result.get('chunk', 'Unknown chunk')}) "
            f"with similarity {float(result.get('similarity', 0)):.2f}"
            for result in context_results
        ]
    )

    # Construct prompt with context
    prompt = f"""You are a helpful AI assistant. 
    Use the following context to answer the query as accurately as possible. If the context is 
    not relevant to the query, say 'I don't know'.

    Context:
    {context_str}

    Query: {query}

    Answer:"""

    # Generate response using Ollama
    response = ollama.chat(
        model=model, messages=[{"role": "user", "content": prompt}]
    )

    return response["message"]["content"]

# Function to detect CPU type
def get_cpu_type():
    cpu_brand = cpuinfo.get_cpu_info()['brand_raw']
    return cpu_brand 

# Function to detect RAM size
def get_ram_size():
    return round(psutil.virtual_memory().total / (1024 ** 3))

# IMPORT THIS
def run_test(queries, embedding_model, llm_model, chunk_size=300, overlap=50):
    redis_client = redis.Redis(host="localhost", port=6379, db=0, decode_responses=True)
    answers = []

    clear_redis_store()
    create_hnsw_index()

    print('Processing PDFs...')
    index_elapsed, index_memory = process_pdfs("All_Slides/", embedding_model, chunk_size, overlap)
    print("\n---Done processing PDFs---\n")

    # define csv file
    csv_filename = "roland_redis_test_results.csv"

    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        
        # Write header only if the file has no data
        if file.tell() == 0:
            writer.writerow(["compute_type", "memory_size", "embedding_model", "llm_model", "index_elapsed", "index_memory", "query", "query_time_elapsed", 'chunk_size', 'overlap'])

        for query in queries:
            print('Query:', query)
            start_time = time.time()
            
            # Generate response
            response = generate_rag_response(query, search_embeddings(query, embedding_model), llm_model)
            print(response)
            answers.append(response)

            elapsed = time.time() - start_time
            print(f'Time elapsed: {round(elapsed, 4)} seconds')
            print('---------------------------')

            cpu_type = get_cpu_type()
            ram_size = get_ram_size()

            # Write data row to CSV
            writer.writerow([cpu_type, ram_size, embedding_model, llm_model, index_elapsed, index_memory, query, round(elapsed, 4), chunk_size, overlap])

    print(f"Results saved to {csv_filename}")

    answers = '\n------------------------\n'.join(answers)
    with open(f"QUERY RESULTS_Redis_{embedding_model.split('/')[1]}_{llm_model.replace('.', '_').replace(':', '_')}_{chunk_size}_{overlap}.txt", 'w') as file:
        file.write(answers)





<>:42: SyntaxWarning: invalid escape sequence '\ '
<>:42: SyntaxWarning: invalid escape sequence '\ '
C:\Users\kibbl\AppData\Local\Temp\ipykernel_33360\1372472663.py:42: SyntaxWarning: invalid escape sequence '\ '
  """
c:\Users\kibbl\anaconda3\envs\captcha\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
c:\Users\kibbl\anaconda3\envs\captcha\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kibbl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kibbl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kibbl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
redis_client = redis.Redis(host="localhost", port=6379, db=0, decode_responses=True)
answers = []

clear_redis_store()
create_hnsw_index()

print('Processing PDFs...')
index_elapsed, index_memory = process_pdfs("All_Slides/", EMBEDDING_MODEL)
print("\n---Done processing PDFs---\n")

Clearing existing Redis store...
Redis store cleared.
Index created successfully.
Processing PDFs...


100%|██████████| 16/16 [08:17<00:00, 31.07s/it]

Time elapsed: 497.1413 seconds
Peak memory usage: 38.58 MiB

---Done processing PDFs---



In [22]:
query = """

Assume you are presented with a collection of documents structured like the following example.  Assume the database is named university and the collection is named students.

{
  "student_id": "S123456789",
  "first_name": "Alex",
  "last_name": "Johnson",
  "email": "alex.johnson@example.edu",
  "enrollment_year": 2023,
  "major": "Computer Science",
  "courses_completed": [
    {
      "course_id": "CS101",
      "course_name": "Introduction to Programming",
      "semester": "Fall 2023",
      "credits": 4,
      "grade": "A-"
    },
    {
      "course_id": "MATH204",
      "course_name": "Calculus II",
      "semester": "Fall 2023",
      "credits": 4,
      "grade": "B+"
    },
    {
      "course_id": "ENG150",
      "course_name": "Academic Writing",
      "semester": "Fall 2023",
      "credits": 3,
      "grade": "A"
    }
  ]
} 

Provide a query to the following prompt using Mongo Query Language

Write a query to return all students who took “Intro to Big Data” in Spring 2023 and got at least a “B+” grade.  Only return the students' first and last names and their email addresses. Return the results sorted by students' last names and then by first names.

"""
response = generate_rag_response(query, search_embeddings(query, EMBEDDING_MODEL), LLM_MODEL)
print(response)



Here is the query you can use to achieve the desired result:

```javascript
db.university.students.find({
  "courses_completed": {
    "$elemMatch": {
      "course_name": "Intro to Big Data",
      "semester": "Spring 2023",
      "grade": { $gte: "B+" }
    }
  }
}, {
  "first_name": 1,
  "last_name": 1,
  "email": 1
}).sort({ last_name: 1, first_name: 1 });
```

This query filters the documents to include only those where a course with name "Intro to Big Data", semester "Spring 2023", and grade at least "B+" exists. It then selects only the `first_name`, `last_name`, and `email` fields for each matching document, and finally sorts the results by last name and then first name in ascending order (1).


In [ ]:
QUERIES = []

for query in QUERIES:
    print(query)
    response = generate_rag_response(query, search_embeddings(query, EMBEDDING_MODEL), LLM_MODEL)
    print(response)
    print('-----------------')